In [1]:
import pandas as pd
import glob
import os
from pyannote.metrics.base import BaseMetric
from inaGVAD.gender_metrics import WstpErr, IdentificationErrorRateLabel
from pyannote.metrics.identification import IdentificationErrorRate

In [2]:
from pyannote.core import Annotation, Timeline, Segment

def df2annot(df, col, rmnan=True, uri=None):
    an = Annotation(uri=uri)
#    print('df2annot', df)
    for start, stop, val in zip(df.start, df.stop, df[col]):
        if rmnan and val != val:
            continue
        seg = Segment(start, stop)
        an[seg] = val
    return an.support()


def init_uem(df):
    uem = Timeline()
    uem.add(Segment(df.start[0], df.stop[len(df) - 1]))
    return uem


def rm_uem(uem, df, col, rmlist):
    for start, stop, val in zip(df.start, df.stop, df[col]):
        if val in rmlist:
            uem = uem.extrude(Segment(start, stop))
    return uem

def keep_uem(uem, df, col, keeplist):
    for start, stop, val in zip(df.start, df.stop, df[col]):
        if val not in keeplist:
            uem = uem.extrude(Segment(start, stop))
    return uem


class GenderEval:
    def __init__(self, collar=.3):
        self.wstp = WstpErr()
        self.ier = IdentificationErrorRate(collar=collar)        
        self.ierF = IdentificationErrorRateLabel('female', collar=collar)
        self.ierM = IdentificationErrorRateLabel('male', collar=collar)
        
    def __call__(self, fref, fpred):
        
        uri, _ = os.path.splitext(os.path.basename(fref))
        
        # parse reference
        dfref = pd.read_csv(fref)
        anref = df2annot(dfref, 'speaker_gender', uri = uri)
        uem = init_uem(dfref)
        uem = rm_uem(uem, dfref, 'speaker_gender', ['undefgender'])
        #print('main uem', uem)
        #print(uem)
        
        # parse prediction
        dfpred = pd.read_csv(fpred)
        #print('dfpred', dfpred)
        if len(dfpred) > 0:
            dfpred = dfpred[dfpred.label.map(lambda x: x in ['male', 'female'])]
        anpred = df2annot(dfpred, 'label', uri = uri)
            
        wstp = self.wstp(anref, anpred, uem=None)
        
        ier = self.ier(anref, anpred, uem=uem)
        errF = self.ierF(anref,anpred,uem=uem)
        errM = self.ierM(anref,anpred,uem=uem)
        
        return wstp, ier, errF, errM
    
    def reset(self):
        self.wstp.reset()
        self.ier.reset()
        self.ierF.reset()
        self.ierM.reset()
        
    def report(self):
        return self.wstp.report(), self.ier.report(), self.ierF.report(), self.ierM.report()
        
    def comparelfiles(self, lref, lhyp):
        for ref, hyp in zip(lref, lhyp):
            print(hyp)
            self(ref, hyp)
        ret = self.report()
        self.reset()
        return ret

ge = GenderEval(collar=0.3)

#
#print(1)
#ge('./annotations/detailed_csv/tv-TF1-131538.csv', './automatic_baselines/inaspeechsegmenter/tv-TF1-131538.csv')
#print(2)
#ge('./annotations/detailed_csv/tv-ART-072411.csv', './automatic_baselines/inaspeechsegmenter/tv-ART-072411.csv')
#ge('./annotations/detailed_csv/tv-TF1-131538.csv', './automatic_baselines/liumspkdirization_csv/tv-TF1-131538.csv')
#ge('./annotations/detailed_csv/tv-ART-072411.csv', './automatic_baselines/liumspkdirization_csv/tv-ART-072411.csv')


In [3]:
ref = './annotations/detailed_csv/'
lref = sorted(glob.glob('./annotations/detailed_csv/*.csv'))
iss = './automatic_baselines/inaspeechsegmenter/'
liss = sorted(glob.glob('./automatic_baselines/inaspeechsegmenter/*.csv'))
lium = './automatic_baselines/liumspkdirization_csv/'
llium = sorted(glob.glob('./automatic_baselines/liumspkdirization_csv/*.csv'))

In [4]:
wstp, ier, ierF, ierM = ge.comparelfiles(lref, liss)

./automatic_baselines/inaspeechsegmenter/radio-FBL-011904.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-012717.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-025455.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-035452.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-044522.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-051706.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-083109.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-090844.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-092729.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-105429.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-132313.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-150454.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-174018.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-205100.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-220700.csv
./automatic_baselines/inaspeechsegmenter/radio-FBL-220712.csv
./automa

./automatic_baselines/inaspeechsegmenter/tv-C+_-112835.csv
./automatic_baselines/inaspeechsegmenter/tv-C+_-134103.csv
./automatic_baselines/inaspeechsegmenter/tv-C+_-141814.csv
./automatic_baselines/inaspeechsegmenter/tv-C+_-165141.csv
./automatic_baselines/inaspeechsegmenter/tv-C+_-175655.csv
./automatic_baselines/inaspeechsegmenter/tv-C+_-181000.csv
./automatic_baselines/inaspeechsegmenter/tv-C+_-190401.csv
./automatic_baselines/inaspeechsegmenter/tv-C+_-203351.csv
./automatic_baselines/inaspeechsegmenter/tv-C+_-204601.csv
./automatic_baselines/inaspeechsegmenter/tv-C+_-220437.csv
./automatic_baselines/inaspeechsegmenter/tv-C25-032841.csv
./automatic_baselines/inaspeechsegmenter/tv-F24-042912.csv
./automatic_baselines/inaspeechsegmenter/tv-F24-045809.csv
./automatic_baselines/inaspeechsegmenter/tv-F24-055841.csv
./automatic_baselines/inaspeechsegmenter/tv-F24-072937.csv
./automatic_baselines/inaspeechsegmenter/tv-F24-075121.csv
./automatic_baselines/inaspeechsegmenter/tv-F24-080416.c

In [5]:
wstp

,women speaking time percentage error,rmale,rfemale,hmale,hfemale
,%,,,,
item,,,,,
radio-FBL-011904,0.000000,0.000,0.000,0.00,0.00
radio-FBL-012717,0.000000,0.000,0.000,0.00,0.00
radio-FBL-025455,0.000000,0.000,0.000,0.00,0.00
radio-FBL-035452,0.000000,0.000,0.000,0.00,0.00
radio-FBL-044522,0.000000,0.000,0.000,0.00,0.00
...,...,...,...,...,...
tv-TF1-215118,-4.688474,5.724,13.731,5.10,15.52
tv-TF1-225026,-5.115000,37.768,8.924,46.10,14.74


In [6]:
ier

identification error rate     total   correct              \
                                         %                               %   
item                                                                         
radio-FBL-011904                  0.000000     0.000     0.000         NaN   
radio-FBL-012717                  0.000000     0.000     0.000         NaN   
radio-FBL-025455                  0.000000     0.000     0.000         NaN   
radio-FBL-035452                  0.000000     0.000     0.000         NaN   
radio-FBL-044522                  0.000000     0.000     0.000         NaN   
...                                    ...       ...       ...         ...   
tv-TF1-215118                    10.443075    16.882    15.965   94.568179   
tv-TF1-225026                    32.358499    43.392    40.673   93.733868   
tv-TFX-230837                    81.186036    24.721    16.571   67.032078   
tv-TFX-231251                    95.639871     7.775     7.775  100.000000   
TOTAL                            13.725407  8989.832  8529.305   94.877246   

                 false alarm            missed detection            confusion  \
                                      %                           %             
item                                                                            
radio-FBL-011904       0.000        NaN            0.000        NaN     0.000   
radio-FBL-012717       0.000        NaN            0.000        NaN     0.000   
radio-FBL-025455       0.000        NaN            0.000        NaN     0.000   
radio-FBL-035452       0.000        NaN            0.000        NaN     0.000   
radio-FBL-044522       0.000        NaN            0.000        NaN     0.000   
...                      ...        ...              ...        ...       ...   
tv-TF1-215118          0.846   5.011255            0.309   1.830352     0.608   
tv-TF1-225026         11.322  26.092367            0.000   0.000000     2.719   
tv-TFX-230837         11.920  48.218114            7.633  30.876583     0.517   
tv-TFX-231251          7.436  95.639871            0.000   0.000000     0.000   
TOTAL                773.364   8.602652          229.869   2.556989   230.658   

                            
                         %  
item                        
radio-FBL-011904       NaN  
radio-FBL-012717       NaN  
radio-FBL-025455       NaN  
radio-FBL-035452       NaN  
radio-FBL-044522       NaN  
...                    ...  
tv-TF1-215118     3.601469  
tv-TF1-225026     6.266132  
tv-TFX-230837     2.091339  
tv-TFX-231251     0.000000  
TOTAL             2.565765  

[278 rows x 10 columns]

In [7]:
ierM

identification error rate     total   correct              \
                                         %                               %   
item                                                                         
radio-FBL-011904                  0.000000     0.000     0.000         NaN   
radio-FBL-012717                  0.000000     0.000     0.000         NaN   
radio-FBL-025455                  0.000000     0.000     0.000         NaN   
radio-FBL-035452                  0.000000     0.000     0.000         NaN   
radio-FBL-044522                  0.000000     0.000     0.000         NaN   
...                                    ...       ...       ...         ...   
tv-TF1-215118                     2.263856     5.124     4.516   88.134270   
tv-TF1-225026                    25.079845    35.068    32.379   92.332041   
tv-TFX-230837                  1745.129225     0.503     0.460   91.451292   
tv-TFX-231251                    95.639871     7.775     7.775  100.000000   
TOTAL                            13.510989  5822.712  5537.030   95.093661   

                 false alarm              missed detection          confusion  \
                                        %                         %             
item                                                                            
radio-FBL-011904       0.000          NaN            0.000      NaN     0.000   
radio-FBL-012717       0.000          NaN            0.000      NaN     0.000   
radio-FBL-025455       0.000          NaN            0.000      NaN     0.000   
radio-FBL-035452       0.000          NaN            0.000      NaN     0.000   
radio-FBL-044522       0.000          NaN            0.000      NaN     0.000   
...                      ...          ...              ...      ...       ...   
tv-TF1-215118          0.116     2.263856            0.000  0.00000     0.000   
tv-TF1-225026          8.765    24.994297            0.000  0.00000     0.030   
tv-TFX-230837          8.304  1650.894632            0.000  0.00000     0.474   
tv-TFX-231251          7.436    95.639871            0.000  0.00000     0.000   
TOTAL                542.106     9.310198          149.812  2.57289    94.788   

                             
                          %  
item                         
radio-FBL-011904        NaN  
radio-FBL-012717        NaN  
radio-FBL-025455        NaN  
radio-FBL-035452        NaN  
radio-FBL-044522        NaN  
...                     ...  
tv-TF1-215118      0.000000  
tv-TF1-225026      0.085548  
tv-TFX-230837     94.234592  
tv-TFX-231251      0.000000  
TOTAL              1.627901  

[278 rows x 10 columns]

In [8]:
ierF

identification error rate     total   correct             \
                                         %                              %   
item                                                                        
radio-FBL-011904                  0.000000     0.000     0.000        NaN   
radio-FBL-012717                  0.000000     0.000     0.000        NaN   
radio-FBL-025455                  0.000000     0.000     0.000        NaN   
radio-FBL-035452                  0.000000     0.000     0.000        NaN   
radio-FBL-044522                  0.000000     0.000     0.000        NaN   
...                                    ...       ...       ...        ...   
tv-TF1-215118                    14.007484    11.758    11.449  97.372002   
tv-TF1-225026                    63.022585     8.324     8.294  99.639596   
tv-TFX-230837                    46.626476    24.218    16.111  66.524899   
tv-TFX-231251                     0.000000     0.000     0.000        NaN   
TOTAL                            14.119610  3167.120  2992.275  94.479369   

                 false alarm            missed detection            confusion  \
                                      %                           %             
item                                                                            
radio-FBL-011904       0.000        NaN            0.000        NaN     0.000   
radio-FBL-012717       0.000        NaN            0.000        NaN     0.000   
radio-FBL-025455       0.000        NaN            0.000        NaN     0.000   
radio-FBL-035452       0.000        NaN            0.000        NaN     0.000   
radio-FBL-044522       0.000        NaN            0.000        NaN     0.000   
...                      ...        ...              ...        ...       ...   
tv-TF1-215118          0.730   6.208539            0.309   2.627998     0.608   
tv-TF1-225026          2.557  30.718405            0.000   0.000000     2.689   
tv-TFX-230837          3.616  14.931043            7.633  31.517879     0.043   
tv-TFX-231251          0.000        NaN            0.000        NaN     0.000   
TOTAL                231.258   7.301839           80.057   2.527754   135.870   

                             
                          %  
item                         
radio-FBL-011904        NaN  
radio-FBL-012717        NaN  
radio-FBL-025455        NaN  
radio-FBL-035452        NaN  
radio-FBL-044522        NaN  
...                     ...  
tv-TF1-215118      5.170947  
tv-TF1-225026     32.304181  
tv-TFX-230837      0.177554  
tv-TFX-231251           NaN  
TOTAL              4.290017  

[278 rows x 10 columns]

In [9]:
wstp, ier, ierF, ierM = ge.comparelfiles(lref, llium)

./automatic_baselines/liumspkdirization_csv/radio-FBL-011904.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-012717.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-025455.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-035452.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-044522.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-051706.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-083109.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-090844.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-092729.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-105429.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-132313.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-150454.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-174018.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-205100.csv
./automatic_baselines/liumspkdirization_csv/radio-FBL-220700.csv
./automatic_baselines/liu

./automatic_baselines/liumspkdirization_csv/tv-C+N-184644.csv
./automatic_baselines/liumspkdirization_csv/tv-C+N-191413.csv
./automatic_baselines/liumspkdirization_csv/tv-C+N-224733.csv
./automatic_baselines/liumspkdirization_csv/tv-C+N-231002.csv
./automatic_baselines/liumspkdirization_csv/tv-C+N-234951.csv
./automatic_baselines/liumspkdirization_csv/tv-C+_-054514.csv
./automatic_baselines/liumspkdirization_csv/tv-C+_-083705.csv
./automatic_baselines/liumspkdirization_csv/tv-C+_-112835.csv
./automatic_baselines/liumspkdirization_csv/tv-C+_-134103.csv
./automatic_baselines/liumspkdirization_csv/tv-C+_-141814.csv
./automatic_baselines/liumspkdirization_csv/tv-C+_-165141.csv
./automatic_baselines/liumspkdirization_csv/tv-C+_-175655.csv
./automatic_baselines/liumspkdirization_csv/tv-C+_-181000.csv
./automatic_baselines/liumspkdirization_csv/tv-C+_-190401.csv
./automatic_baselines/liumspkdirization_csv/tv-C+_-203351.csv
./automatic_baselines/liumspkdirization_csv/tv-C+_-204601.csv
./automa

./automatic_baselines/liumspkdirization_csv/tv-TFX-231251.csv


In [10]:
wstp

,women speaking time percentage error,rmale,rfemale,hmale,hfemale
,%,,,,
item,,,,,
radio-FBL-011904,0.000000,0.000,0.000,0.00,0.00
radio-FBL-012717,50.000000,0.000,0.000,17.69,0.00
radio-FBL-025455,0.000000,0.000,0.000,0.00,0.00
radio-FBL-035452,-50.000000,0.000,0.000,0.00,7.75
radio-FBL-044522,50.000000,0.000,0.000,14.12,0.00
...,...,...,...,...,...
tv-TF1-215118,53.595289,5.724,13.731,34.12,6.98
tv-TF1-225026,2.778031,37.768,8.924,47.43,9.26


In [11]:
ier

identification error rate     total   correct             \
                                         %                              %   
item                                                                        
radio-FBL-011904                  0.000000     0.000     0.000        NaN   
radio-FBL-012717                100.000000     0.000     0.000        NaN   
radio-FBL-025455                  0.000000     0.000     0.000        NaN   
radio-FBL-035452                100.000000     0.000     0.000        NaN   
radio-FBL-044522                100.000000     0.000     0.000        NaN   
...                                    ...       ...       ...        ...   
tv-TF1-215118                   183.485369    16.882     8.172  48.406587   
tv-TF1-225026                    20.349373    43.392    42.508  97.962758   
tv-TFX-230837                    89.057886    24.721    19.836  80.239473   
tv-TFX-231251                   168.372990     7.775     0.000   0.000000   
TOTAL                            36.038449  8989.832  7799.273  86.756605   

                 false alarm             missed detection             \
                                       %                           %   
item                                                                   
radio-FBL-011904       0.000         NaN            0.000        NaN   
radio-FBL-012717      17.690         NaN            0.000        NaN   
radio-FBL-025455       0.000         NaN            0.000        NaN   
radio-FBL-035452       7.750         NaN            0.000        NaN   
radio-FBL-044522      14.120         NaN            0.000        NaN   
...                      ...         ...              ...        ...   
tv-TF1-215118         22.266  131.891956            0.965   5.716147   
tv-TF1-225026          7.946   18.312131            0.474   1.092367   
tv-TFX-230837         17.131   69.297359            4.261  17.236358   
tv-TFX-231251          5.316   68.372990            1.021  13.131833   
TOTAL               2049.237   22.795053          338.394   3.764186   

                 confusion             
                                    %  
item                                   
radio-FBL-011904     0.000        NaN  
radio-FBL-012717     0.000        NaN  
radio-FBL-025455     0.000        NaN  
radio-FBL-035452     0.000        NaN  
radio-FBL-044522     0.000        NaN  
...                    ...        ...  
tv-TF1-215118        7.745  45.877266  
tv-TF1-225026        0.410   0.944875  
tv-TFX-230837        0.624   2.524170  
tv-TFX-231251        6.754  86.868167  
TOTAL              852.165   9.479209  

[278 rows x 10 columns]

In [12]:
ierF

identification error rate     total   correct             \
                                         %                              %   
item                                                                        
radio-FBL-011904                  0.000000     0.000     0.000        NaN   
radio-FBL-012717                  0.000000     0.000     0.000        NaN   
radio-FBL-025455                  0.000000     0.000     0.000        NaN   
radio-FBL-035452                100.000000     0.000     0.000        NaN   
radio-FBL-044522                  0.000000     0.000     0.000        NaN   
...                                    ...       ...       ...        ...   
tv-TF1-215118                    26.560640    11.758     3.795  32.275897   
tv-TF1-225026                    18.428640     8.324     7.520  90.341182   
tv-TFX-230837                    69.654802    24.218    19.836  81.906020   
tv-TFX-231251                   100.000000     0.000     0.000        NaN   
TOTAL                            55.461997  3167.120  2807.678  88.650825   

                 false alarm            missed detection            confusion  \
                                      %                           %             
item                                                                            
radio-FBL-011904       0.000        NaN            0.000        NaN     0.000   
radio-FBL-012717       0.000        NaN            0.000        NaN     0.000   
radio-FBL-025455       0.000        NaN            0.000        NaN     0.000   
radio-FBL-035452       7.750        NaN            0.000        NaN     0.000   
radio-FBL-044522       0.000        NaN            0.000        NaN     0.000   
...                      ...        ...              ...        ...       ...   
tv-TF1-215118          1.411  12.000340            0.965   8.207178     0.747   
tv-TF1-225026          1.140  13.695339            0.394   4.733301     0.000   
tv-TFX-230837         12.191  50.338591            4.218  17.416797     0.460   
tv-TFX-231251          5.316        NaN            0.000        NaN     6.754   
TOTAL                986.937  31.161970          138.444   4.371290   631.167   

                             
                          %  
item                         
radio-FBL-011904        NaN  
radio-FBL-012717        NaN  
radio-FBL-025455        NaN  
radio-FBL-035452        NaN  
radio-FBL-044522        NaN  
...                     ...  
tv-TF1-215118      6.353121  
tv-TF1-225026      0.000000  
tv-TFX-230837      1.899414  
tv-TFX-231251           NaN  
TOTAL             19.928737  

[278 rows x 10 columns]

In [13]:
ierM

identification error rate     total   correct             \
                                         %                              %   
item                                                                        
radio-FBL-011904                  0.000000     0.000     0.000        NaN   
radio-FBL-012717                100.000000     0.000     0.000        NaN   
radio-FBL-025455                  0.000000     0.000     0.000        NaN   
radio-FBL-035452                  0.000000     0.000     0.000        NaN   
radio-FBL-044522                100.000000     0.000     0.000        NaN   
...                                    ...       ...       ...        ...   
tv-TF1-215118                   543.579235     5.124     4.377  85.421546   
tv-TF1-225026                    20.805293    35.068    34.988  99.771872   
tv-TFX-230837                  1023.260437     0.503     0.000   0.000000   
tv-TFX-231251                    13.131833     7.775     0.000   0.000000   
TOTAL                            25.473491  5822.712  4991.595  85.726290   

                 false alarm             missed detection             \
                                       %                           %   
item                                                                   
radio-FBL-011904       0.000         NaN            0.000        NaN   
radio-FBL-012717      17.690         NaN            0.000        NaN   
radio-FBL-025455       0.000         NaN            0.000        NaN   
radio-FBL-035452       0.000         NaN            0.000        NaN   
radio-FBL-044522      14.120         NaN            0.000        NaN   
...                      ...         ...              ...        ...   
tv-TF1-215118         20.855  407.006245            0.000   0.000000   
tv-TF1-225026          6.806   19.408007            0.080   0.228128   
tv-TFX-230837          4.940  982.107356            0.043   8.548708   
tv-TFX-231251          0.000    0.000000            1.021  13.131833   
TOTAL               1062.300   18.244076          199.950   3.433967   

                 confusion              
                                     %  
item                                    
radio-FBL-011904     0.000         NaN  
radio-FBL-012717     0.000         NaN  
radio-FBL-025455     0.000         NaN  
radio-FBL-035452     0.000         NaN  
radio-FBL-044522     0.000         NaN  
...                    ...         ...  
tv-TF1-215118        6.998  136.572990  
tv-TF1-225026        0.410    1.169157  
tv-TFX-230837        0.164   32.604374  
tv-TFX-231251        0.000    0.000000  
TOTAL              220.998    3.795448  

[278 rows x 10 columns]